In [ ]:
# MICROBIOME 16S PIPELINE ION TORRENT.

# This pipeline was developed for microbiome 16S data analysis comming from Ion Torrent technology (Ion 16S™ Metagenomics Kit) in the Hematological Tumor Unit of
# Universidad Complutense de Madrid/Hospital Doce de Octubre/CNIO.

# BEFORE STARTING:

# A. Ion 16S™ Metagenomics Kit uses 2 pools of primers (Pool 1: V2, V4, V8 and Pool 2: V3, V6-7 and V9) and it generates a FASTQ file per sample which contains sequences 
# comming from all V regions. This kit uses bidireccional sequencing (not paired-end) with forward (F) and reverse (R) primers, so both sequence types are included in 
# each FASTQ file. In addition, sequences are demultiplexed and molecular barcodes are removed by the sequence machine. PRIMERS SEQUENCES ARE PROPIETARY AND UNKNOWN!!!

# B. CURRENT LIMITATIONS Ion 16S™ Metagenomics Kit: Although the Ion Torrent software used to analyse data (Ion Reporter) adds up OTUs comming from the different V regions,
# this process has important bias and it is not considered as suitable by a high number of Qiime2 and Ion Torrent users. The key is that V primers don't overlap between them
# so this data can't be treated as other omics data (e.g RNA-seq counts) which are assembled before counting. In this case, if OTUs from different V regions are added up, we
# will overestimate the richness in our community because some taxa will be count double, triple or more!!! IF THIS APPROACH IS FOLLOWED, THIS ASPECT SHOULD BE MENTIONED IN 
# THE DISCUSSION SPECIFYING THAT THERE IS NOT A SUITABLE APPROACH FOR MERGING ACCROSS V REGIONS AT DATE OF MARCH 2023. 

# C. To cover the great disadvantage of this kit, this pipeline will generate outputs for all individual V region and for a "consensus" (created as Ion Reporter)

# THIS CODE WAS USED TO REORGANIZE FASTQ FILES BY V REGION (PREVIOUSLY DIVIDED BY SAMPLE AND V REGION), GENERATE COUNTS STATISTICS FOR EACH V REGION AND 
# WRITE A MANIFEST FILE TO IMPORT FASTQ FILES FOR EACH V REGION IN QIIME2.

# This code was generated by Katherine Maki and partners (add citation in the paper)
import os
import shutil
import gzip
import pandas as pd # version 1.4.0

### move or copy fastq or fastq.gz files to a new directory structure
def reorg_metagenomics_files(oldPath='./cutprimers/cutprimers_output',
                             newPath='./cutprimers/cutprimers_Vsorted'):
    if not os.path.isdir(newPath):
        os.mkdir(newPath)
        
    ## Read file tree in oldPath and get names of fastq files
    def getFileInfo(oldPath,newPath):
        filesOld,filesNew = [],[]
        v_regions = set()   
        for root, dirs, filenames in os.walk(oldPath):
            for file in filenames:
                if file[-6:].lower() == '.fastq' or file[-9:].lower() == '.fastq.gz':
                    filesOld.append(os.path.join(root,file))
                    f = file.replace('FR.','')
                    v = f.split('.')[1]
                    v_regions.add(v)
                    fNew = f.replace('.' + v,'_' + v)
                    filesNew.append(os.path.join(newPath,v,fNew))
        return filesOld,filesNew,v_regions
    filesOld,filesNew,v_regions = getFileInfo(oldPath,newPath)
    assert len(filesOld), 'no files found in "{}"'.format(oldPath)
    
    ## prompt for user input before manipulating files
    def getUserInputs(v_regions):     
        print('\nSubdirectories:')
        for v in sorted(v_regions):
            print('\t{}'.format(v))
            
        inputs = {}
        def getInput(dictionary,key,msg):
            while dictionary.get(key) not in {'y','n'}:
                dictionary[key] = input(msg)
        getInput(inputs,'copy','Copy fastq files into the above subdirectories? [y/n] ')
        if inputs['copy'] == 'n':
            getInput(inputs,'move','Move fastq files into the above subdirectories? [y/n] ')
        else:
            getInput(inputs,'gzip','Compress (gzip) the new fastq files? [y/n] ')
        print()
        return inputs
    inputs = getUserInputs(v_regions)
        
    ## if compressing files, then add any missing .gz extension to names of new files
    def gzipFileNames(files):
        return [f  + '.gz' if f[-3:] != '.gz' else f for f in files]
    if inputs.get('gzip') == 'y':
        filesNew = gzipFileNames(filesNew)
    
    ## copy fastq files, optionally use gzip to compress any uncompressed files
    def copyFiles(filesOld,filesNew,compress='n'):
        if compress == 'n':
            print('\t{} files total, copying file     '.format(len(filesNew)),end='')
            for i,(fOld,fNew) in enumerate(zip(filesOld,filesNew)):
                print('\b\b\b\b{0:4d}'.format(i+1),end='')
                os.makedirs(os.path.dirname(fNew),exist_ok=True)
                shutil.copy2(fOld,fNew)
            print('\n\tAll fastq files copied!\n')
        else:
            print('\t{} files total, compressing file    '.format(len(filesNew)),end='')
            for i,(fOld,fNew) in enumerate(zip(filesOld,filesNew)):
                print('\b\b\b\b{0:4d}'.format(i+1),end='')
                os.makedirs(os.path.dirname(fNew),exist_ok=True)
                if fNew[-3:] == '.gz' and fOld[-3:] != '.gz':
                    with open(fOld,'rb') as f:
                        with gzip.open(fNew,'wb') as fzip:
                            shutil.copyfileobj(f,fzip)
                else:
                     shutil.copy2(fOld,fNew)
            print('\n\tAll fastq files copied as .gz files!\n')
            
    ## move files to new location -- obviously faster than copying
    def moveFiles(filesOld,filesNew):
        for fOld,fNew in zip(filesOld,filesNew):
            os.makedirs(os.path.dirname(fNew),exist_ok=True)
            shutil.move(fOld,fNew)
            
    ## execute move or copy        
    if inputs.get('move') == 'y':
        moveFiles(filesOld,filesNew)
    elif inputs['copy'] == 'y':
        copyFiles(filesOld,filesNew,inputs['gzip'])
    
    ## write aggregate counts file
    def aggregate_counts_data(writeFile,oldPath,stats_rows=True):
        # Aggregates total counts from "counts.txt" files in the mainPath directory 
        # tree. File has a row for each sample and a column for each primer.
        # If stats_rows is True, then it will add rows of statistics to the end.
        
        dfs = []
        for root, dirs, filenames in os.walk(oldPath):
            for file in filenames:
                if file == 'counts.txt':
                    dfs.append(pd.read_csv(os.path.join(root,file),sep='\t',
                                           names=['name','count']))
        df = pd.concat(dfs)
        df = df[df['name'] != 'total']
        df = pd.concat((df.name.str.split('.',expand=True),df['count']),axis=1)
        df = df.pivot_table(index=0,columns=1,values='count')
        if stats_rows:
            df = pd.concat((df,df.agg(['mean','std','sem'])))
        df.to_csv(writeFile)

    countsFile = os.path.join(newPath,'total_counts.csv')
    aggregate_counts_data(countsFile,oldPath,stats_rows=True)
    print('\tAggregate counts file written to {}\n'.format(countsFile))   
    
### write manifest files for QIIME2
def writeManifestFiles(newPath='./cutprimers/cutprimers_Vsorted',
                       manifestPath='/data/NRTS_share/ion_torrent_qiime2_methods/'):
    
    def getFastqFiles(path):
        files = []
        v_regions = set()
        for root, dirs, filenames in os.walk(path):
            for file in filenames:
                if file[-6:].lower() == '.fastq' or file[-9:].lower() == '.fastq.gz':
                    files.append(os.path.join(root,file))
                    v_regions.add(root.split(os.sep)[-1])
        return [os.path.normpath(f) for f in files],v_regions

    ### write manifest files
    def writeRegionManifest(writeFile,filesNew,v,sep='\t'):  #tab:'\t' 
        # writeFile has two columns: first is the sample ID, next is the absolute path
        # of the file on the biowulf cluster
        files = [f for f in filesNew if v + '.' in f]
        fp = [(manifestPath+f).replace(os.sep,'/') for f in files]
        sID = [os.path.basename(f).split('.')[0] for f in files]    
        manifest = pd.DataFrame({'sample-id': sID, 'absolute-filepath': fp})
        manifest.to_csv(writeFile,index=False,sep=sep)
        print('\tManifest file written to {}'.format(manifestFile))
    
    ## execute functions
    filesNew,v_regions = getFastqFiles(newPath)
    # print(v_regions)
    for v in sorted(v_regions):
        manifestFile = os.path.join(newPath,'manifest_{}.txt'.format(v))
        writeRegionManifest(manifestFile,filesNew,v)
    print()

#### 0. CHECK METADATA RENAME ID FILES

In [ ]:
%%bash

# IMPORTANT: Set path to rename_ids metadata information in each project and create the first "v2_ids.tsv" (this exact name) each time.
cd ~/Escritorio/bioinfo_hematology_unit/tfm_david/metadata

# We have an V2 file wich maps samples IDs generated by MetagenomicsPP pipeline (sample_name_V2) with the same sample name whitout the V region ending (_V2),
# which is necessary to work with q2-sidle.

# Duplicate the files for the rest V regions.

# V3
sed -ie 's/V2/V3/g' v2_ids.tsv

# V4
sed -ie 's/V2/V4/g' v2_ids.tsve

# V67
sed -ie 's/V2/V67/g' v2_ids.tsvee

# V8
sed -ie 's/V2/V8/g' v2_ids.tsveee

# V9
sed -ie 's/V2/V9/g' v2_ids.tsveeee

# Change file names to make them usable.
mv v2_ids.tsv v3_ids.tsv
mv v2_ids.tsve v4_ids.tsv
mv v2_ids.tsvee v67_ids.tsv
mv v2_ids.tsveee v8_ids.tsv
mv v2_ids.tsveeee v9_ids.tsv
mv v2_ids.tsveeeee v2_ids.tsv

#### 1. SPLIT FASTQ FILES BY V REGION IN EACH SAMPLE.


In [ ]:
%%bash
# This is developed using a plugin from Thermofisher called MetagenomicsPP. It generates a new folder with a number of subfolder equal to the name of samples (fastq). In each 
# subfolder, it generetes an independent fastq file for each V region, another fastq file for non-adapters sequeces and summary statistics.

# Note: with cutprimers, both the forward and reverse reads are combined into one "V" region file, and the reverse are inverse complemented so all are intepreted as forward reads.

#! cd ~/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/

sh ~/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/MetagenomicsPP/mgpp.sh -m ~/Escritorio/bioinfo_hematology_unit/tfm_david/data/manifest_emumyc.txt -o ~/Escritorio/bioinfo_hematology_unit/tfm_david/data/mgpp_results


#### 2. REORGANIZE FASTQ FILES BY V REGION


In [ ]:
reorg_metagenomics_files(oldPath = '/home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/mgpp_results',
                         newPath = '/home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted')

#### 3. CREATE A MANIFEST FILE FROM THE SUB-V FOLDERS IN QIIME2.

In [ ]:
# manifestPath = "": To save manifest.txt in the same folder as fastq files.
writeManifestFiles(newPath = "/home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted",
                   manifestPath = "")

#### 4. IMPORT SEPARATED FASTQ FILES INTO QIIME2.

In [ ]:
# Input fomart is "SingleEndFastqManifestPhred33V2" because reverse sequences were inverse complemented

# IMPORTANT: IT WILL BE NEEDED TO DISGUISH BETWEEN RUNS IN THE FUTURE. FOR NOW, WE ONLY HAVE ONE RUN.

# V2 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V2.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/demux_seqs/demux_seqs_V2.qza

# V3 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V3.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/demux_seqs/demux_seqs_V3.qza

# V4 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V4.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/demux_seqs/demux_seqs_V4.qza

# V67 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V67.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/demux_seqs/demux_seqs_V67.qza

# V8 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V8.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/demux_seqs/demux_seqs_V8.qza

# V9 data
! qiime tools import --type "SampleData[SequencesWithQuality]" \
    --input-format SingleEndFastqManifestPhred33V2 \
    --input-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/fastq_Vsorted/manifest_V9.txt \
    --output-path /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/demux_seqs/demux_seqs_V9.qza


#### 5. VISUALIZE QUALITY.


##### 5.1 V2

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/demux_seqs/demux_seqs_V2.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/demux_seqs/demux_seqs_V2.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/demux_seqs/demux_seqs_V2.qzv

##### 5.2 V3

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/demux_seqs/demux_seqs_V3.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/demux_seqs/demux_seqs_V3.qzv 

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/demux_seqs/demux_seqs_V3.qzv 

##### 5.3 V4

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/demux_seqs/demux_seqs_V4.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/demux_seqs/demux_seqs_V4.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/demux_seqs/demux_seqs_V4.qzv

##### 5.4 V67

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/demux_seqs/demux_seqs_V67.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/demux_seqs/demux_seqs_V67.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/demux_seqs/demux_seqs_V67.qzv

##### 5.5 V8

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/demux_seqs/demux_seqs_V8.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/demux_seqs/demux_seqs_V8.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/demux_seqs/demux_seqs_V8.qzv

##### 5.6 V9

In [ ]:
%%bash
qiime demux summarize \
  --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/demux_seqs/demux_seqs_V9.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/demux_seqs/demux_seqs_V9.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/demux_seqs/demux_seqs_V9.qzv

#### 6. DADA2 with denoise -pyro


As in the previous step, this must be used with each run separately and for each V region.

##### 6.1 V2

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/demux_seqs/demux_seqs_V2.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qza \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v2_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qza

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_stats.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_stats.qzv

# Visualize seqs
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qzv

##### 6.2 V3

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/demux_seqs/demux_seqs_V3.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qza \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v3_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qza

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_stats.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_stats.qzv 

# Visualize seqs
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qzv

##### 6.3 V4

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/demux_seqs/demux_seqs_V4.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qza  \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v4_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qza 

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_stats.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_stats.qzv

# Visualize seqs
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qzv

##### 6.4 V67

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/demux_seqs/demux_seqs_V67.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qza  \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v67_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qza

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_stats.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_stats.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qzv

##### 6.5 V8

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/demux_seqs/demux_seqs_V8.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qza  \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v8_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qza

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_stats.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_stats.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qzv

##### 6.6 V9

In [ ]:
%%bash

# HERE, WE USED 170 AS EXAMPLE
qiime dada2 denoise-pyro \
--p-trunc-len 170  \
--i-demultiplexed-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/demux_seqs/demux_seqs_V9.qza  \
--o-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qza \
--o-representative-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_rep_seq.qza \
--o-denoising-stats /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_stats.qza \
--verbose

In [ ]:
%%bash

# Rename sample ID from "name_VX" to "name".
qiime feature-table rename-ids \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qza \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/rename_ids_metadata/v9_ids.tsv \
    --m-metadata-column 'NewID' \
    --p-axis 'sample' \
    --o-renamed-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qza

In [ ]:
%%bash

# Visualize stats
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_stats.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_stats.qzv

# Visualize feature table
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_stats.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qzv

### IMPORTANT: If q2-sidle works properly, we won't need to combine feature tables and representative sequences accross V regions and we would directly obtain a feature table and a reconstructed taxonomy.

### 7. q2-sidle

#### 7.1 Database preparation.

This step will need to be done once for each **database**, **primer pair** and **kmer lenght**, since the reference files can be reused.

In our case, **primer pairs** will be the same because we'll use the same Ion Torrent kit for further experiments. Database will be SILVA 128 until the last version of SILVA (138 or upper) can be used with q2-sidle. So, **database preparation will need to be repeatead each time a new specific kmer obtained with *dada2-denoise pyro* was set**.

##### 7.1.1 Get SILVA 128 database.

In [ ]:
%%bash

# Get SILVA 128 seqs and taxonomy
qiime rescript get-silva-data \
    --p-version '128' \
    --p-target 'SSURef_NR99' \
    --p-include-species-labels \
    --o-silva-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_rna_seqs.qza \
    --o-silva-taxonomy /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax.qza

In [ ]:
%%bash

# Transcribe RNA sequences to DNA.
qiime rescript reverse-transcribe \
    --i-rna-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_rna_seqs.qza \
    --o-dna-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs.qza 

In [ ]:
%%bash

# Visualize both artifacts
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_seqs.qzv

In [ ]:
%%bash
qiime metadata tabulate \
   --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax.qza \
   --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_tax.qzv 

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_seqs.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_tax.qzv

##### 7.1.2 “Culling” low-quality sequences with cull-seqs.


In [ ]:
%%bash

# Number degenerates nucleotides: 5.
qiime rescript cull-seqs \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs.qza \
    --p-n-jobs 4 \
    --o-clean-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned.qza

In [ ]:
%%bash

# Get SILVA 128 cleaned visualization. 612 514 total sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_seqs_cleaned.qzv

##### 7.1.3  Filtering sequences by lenght and taxonomy.


In [ ]:
%%bash

# It allows us to remove low quality regions.
qiime rescript filter-seqs-length-by-taxon \
 --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned.qza \
 --i-taxonomy /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax.qza \
 --p-labels Archaea Bacteria Eukaryota \
 --p-min-lens 900 1200 1400 \
 --o-filtered-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt.qza \
 --o-discarded-seqs /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_discarded.qza

In [ ]:
%%bash

# Get SILVA 128 cleaned filtered (length and taxonomy) visualization. 607415 total sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_seqs_cleaned_filt.qzv

##### 7.1.4  Dereplication.


In [ ]:
%%bash

# Dereplication using the default "uniq" approach.
 qiime rescript dereplicate \
  --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt.qza \
  --i-taxa /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax.qza \
  --p-mode 'uniq' \
  --p-perc-identity 0.99 \
  --p-threads 5 \
  --p-derep-prefix \
  --o-dereplicated-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep.qza \
  --o-dereplicated-taxa /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax_derep.qza

In [ ]:
%%bash

# Get SILVA 128 cleaned, filtered and dereplicated visualization. 392 994 sequences (36% filtered out)
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/visualizations/silva_128_ssu_nr99_seqs_cleaned_filt_derep.qzv

##### 7.1.5  Filtering sequences by taxonomy.


In [ ]:
%%bash

# Filter by taxonomy
qiime taxa filter-seqs \
 --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep.qza \
 --i-taxonomy /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_tax_derep.qza \
 --p-include 'Bacteria, Archea' \
 --p-exclude 'p__;,k__;,Chloroplast,c__Chloroplast;, Mitochondria,f__mitochondria;, Unassigned' \
 --o-filtered-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza

In [ ]:
%%bash

# Get SILVA 128 cleaned, filtered by quality, dereplicated and filterd by taxonomy visualization.  328 454 sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qzv

##### 7.1.6  Prepare a regional database for each primer set.


**A. Extracting reads**

In [ ]:
%%bash

# V2
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer NGCGNACGGGTGAGTAA \
    --p-r-primer GCTGCCTCCCGTAGGAG \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V2.qza

In [ ]:
%%bash

# V2 database visualization. 310 427 sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V2.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V2.qzv

In [ ]:
%%bash

# V3
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer CTGAGACACGGTCCANACT \
    --p-r-primer GTATTACCGCGGCTGCTG \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V3.qza

In [ ]:
%%bash

# V3 database visualization. 326 065 sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V3.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V3.qzv

In [ ]:
%%bash

# V4
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer CCAGCAGCCGCGGTAATA \
    --p-r-primer GGACTACCAGGGTATCTAATCCTGT \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V4.qza

In [ ]:
%%bash

# V4 database visualization. 326 851 sequences.
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V4.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V4.qzv

In [ ]:
%%bash

# V67
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer ACAAGCGGNGGAGCATGT \
    --p-r-primer GACGTCATCCCCACCTTCC \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V67.qza

In [ ]:
%%bash

# V67 database visualization. 322 578 sequeneces (211-234 bp)
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V67.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V67.qzv

In [ ]:
%%bash

# V8
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer GNTGTCGTCAGCTCGTGT \
    --p-r-primer CGATTACTAGCGANTCCGACTTCA \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V8.qza

In [ ]:
%%bash

# V8 database visualization. 320 902 (233-262)
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V8.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V8.qzv

In [ ]:
%%bash

# V9
qiime feature-classifier extract-reads \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/silva_128_ssu_nr99_seqs_cleaned_filt_derep_filttax.qza \
    --p-f-primer GTTACGACTTCACCCCAGTCA \
    --p-r-primer GCGTCGTAGTCCGGATTGG \
    --p-n-jobs 4 \
    --o-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V9.qza

In [ ]:
%%bash

# V8 database visualization. 73 952 (161-919)
qiime feature-table tabulate-seqs \
 --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V9.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V9.qzv

**B. Extracting region**

THIS STEP SHOULD BE REPEATED EACH TIME WE CHANGE THE TRIMMING/TRUNCATING PARAMETERS DURING DENOYSING.

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V2
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V2.qza \
    --p-region "V2" \
    --p-fwd-primer NGCGNACGGGTGAGTAA \
    --p-rev-primer GCTGCCTCCCGTAGGAG \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V2_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V2_170nt_map.qza

In [ ]:
%%bash

# Visualize V2 Kmer map.
qiime metadata tabulate \
 --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V2_170nt_map.qza \
 --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V2_170nt_map.qzv

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V3
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V3.qza \
    --p-region "V3" \
    --p-fwd-primer CTGAGACACGGTCCANACT \
    --p-rev-primer GTATTACCGCGGCTGCTG \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V3_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V3_170nt_map.qza

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V4
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V4.qza \
    --p-region "V4" \
    --p-fwd-primer CCAGCAGCCGCGGTAATA \
    --p-rev-primer GGACTACCAGGGTATCTAATCCTGT \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V4_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V4_170nt_map.qza

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V67
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V67.qza \
    --p-region "V67" \
    --p-fwd-primer ACAAGCGGNGGAGCATGT \
    --p-rev-primer GACGTCATCCCCACCTTCC \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V67_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V67_170nt_map.qza

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V8
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V8.qza \
    --p-region "V8" \
    --p-fwd-primer GNTGTCGTCAGCTCGTGT \
    --p-rev-primer CGATTACTAGCGANTCCGACTTCA \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V8_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V8_170nt_map.qza

In [ ]:
%%bash

# READS WERE TRIMMED AS 170 LIKE OUR SEQUENCING READS.

# V9
qiime sidle prepare-extracted-region \
    --i-sequences /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/silva_128_ssu_nr99_V9.qza \
    --p-region "V9" \
    --p-fwd-primer GTTACGACTTCACCCCAGTCA \
    --p-rev-primer GCGTCGTAGTCCGGATTGG \
    --p-trim-length 170 \
    --p-n-workers 4 \
    --o-collapsed-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V9_170nt.qza \
    --o-kmer-map /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V9_170nt_map.qza

Here, we have our **regional databases for each V region**  with sequences trimmed at **170 nt** as the asv sequences prepared with denoise-pyro.

#### 7.2  Read preparation.

For this step, we have the feature tables and representative sequences (ASV) for each V region. It is important to check:

- The trimmed length is the same for representative sequences and regional databases on a per V region way. For instace, if sequences comming from V4 were trimmed at 160 nt during denoise-pyro step, the sequeneces in the SILVA 128 V4 regional database must have the same lenght.

- All samples must have the same names in the feature tables for each V region.

#### 7.3 Sequence Reconstruction.

##### 7.3.1 Regional alignment.

One ASV sequence can map with more than one kmer sequence from the regional database. So, the number of aligments is higher than the number of ASVs generated by denoise-pyro.

**V2 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V2
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V2_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qza \
    --p-region "V2" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V2_aligment_map.qza \

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V2_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V2_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V2_aligment_map.qzv

**V3 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V3
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V3_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qza \
    --p-region "V3" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V3_aligment_map.qza

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V3_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V3_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V3_aligment_map.qzv

**V4 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V4
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V4_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qza \
    --p-region "V4" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V4_aligment_map.qza

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V4_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V4_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V4_aligment_map.qzv

**V67 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V67
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V67_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_rep_seq.qza \
    --p-region "V67" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V67_aligment_map.qza

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V67_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V67_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V67_aligment_map.qzv

**V8 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V8
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V8_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_rep_seq.qza \
    --p-region "V8" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V8_aligment_map.qza

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V8_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V8_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V8_aligment_map.qzv

**V9 regional aligment**

In [ ]:
%%bash

# MAX_MISMATCH should be adjusted according to kmer and asv lengths.

# V9
qiime sidle align-regional-kmers \
    --i-kmers /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170/silva_128_V9_170nt.qza \
    --i-rep-seq /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_rep_seq.qza \
    --p-region "V9" \
    --p-max-mismatch 3 \
    --p-n-workers 4 \
    --o-regional-alignment /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V9_aligment_map.qza

In [ ]:
%%bash

# Visualize kmer aligment
qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V9_aligment_map.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V9_aligment_map.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/regional_alignments_sidle/V9_aligment_map.qzv

##### 7.3.2 Database, Feature Table and Taxonomy Reconstruction.

In [ ]:
if __name__ == '__main__':
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster(n_workers=2, memory_limit = "8GB")
    client = Client(cluster)

In [ ]:
import ctypes

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

client.run(trim_memory)

In [ ]:
cluster.scheduler

In [ ]:
cluster.workers

In [ ]:
client

In [ ]:
client.scheduler_info()

In [ ]:
client.restart()

In [ ]:
%%bash

cd /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/128/artifacts/regional_databases/kmer_170

qiime sidle reconstruct-database \
    --i-regional-alignment V2_aligment_map.qza V3_aligment_map.qza V4_aligment_map.qza V67_aligment_map.qza V8_aligment_map.qza V9_aligment_map.qza \
    --i-kmer-map silva_128_V2_170nt_map.qza silva_128_V3_170nt_map.qza silva_128_V4_170nt_map.qza silva_128_V67_170nt_map.qza silva_128_V8_170nt_map.qza silva_128_V9_170nt_map.qza \
    --p-region V2 V3 V4 V67 V8 V9 \
    --p-block-size 1000 \
    --p-client-address 'tcp://127.0.0.1:32915' \
    --o-database-map ./reconstructed_results/database_recons.qza \
    --o-database-summary ./reconstructed_results/database_recons_summ.qza \
    --verbose

#### 7. Combining different DADA2 V regions.


Feature tables per V region could be used to develop a comparison between V regions in terms of performance.

**IMPORTANT**: If there were multiple runs, we would need to combine run specific results (feature table and representative sequences) and, then, further combination accross V regions to generate consensus results. 

Ej: 3 runs with 10 samples each one for the V2, V3, V4, V67, v8 and V9 regions.

- **1º Combine V regions accross runs**: 

        V2_run1 + V2_run2 + V2_run3 = V2_merged

        V3_run1 + V3_run2 + V3_run3 = V3_merged

        V4_run1 + V4_run2 + V4_run3 = V4_merged

        V67_run1 + V67_run2 + V67_run3 = V67_merged

        V8_run1 + V8_run2 + V8_run3 = V8_merged

        V9_run1 + V9_run2 + V9_run3 = V9_merged

- **2º Combine accross V regions to generate consensus results (NOT RECOMMENDED)**:

        V2_merged + V3_merged + V4_merged + V67_merged + V8_merged + V9_merged = consensus_results
        

As in this case, we only have one run, sequences were merged accros V regions.

In [ ]:
%%bash

qiime feature-table merge \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_table.qza \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_table.qza \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_table.qza \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_table.qza \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_table.qza \
    --i-tables /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_table.qza \
    --o-merged-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_sampleid_table.qza \
    --p-overlap-method sum
    

In [ ]:
%%bash

qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_sampleid_table.qza \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_sampleid_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_sampleid_table.qzv

In [ ]:
from qiime2 import Metadata
metadata_vregions = Metadata.load("/home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/metadata_vregions.tsv")
metadata_vregions = metadata_vregions.to_dataframe()
metadata_vregions

In [ ]:
%%bash
# Group sampleid by patientid

qiime feature-table group \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_sampleid_table.qza \
    --p-axis 'sample' \
    --m-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/metadata_vregions.tsv \
    --m-metadata-column 'PatientID' \
    --p-mode sum \
    --o-grouped-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_feature_table.qza

In [ ]:
%%bash

qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_feature_table.qza\
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_feature_table.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_feature_table.qzv

**Feature Table Summary**: In this step, the main characteristics of the "consensus" feature table are summarized.

IMPORTANT: This summary could be done with the feature tables per V region (feature tables merged for each V region) to identify how the performance differs between V regions.

**Invented Metadata**

In [ ]:
from qiime2 import Metadata
metadata = Metadata.load("/home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/metadata_example.tsv")
metadata = metadata.to_dataframe()
metadata

In [ ]:
%%bash

qiime feature-table summarize \
    --i-table /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_feature_table.qza \
    --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_ft_summary.qzv \
    --m-sample-metadata-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/metadata/metadata_example.tsv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_ft_summary.qzv

#### 8. Combining different DADA2 sequences.

In this case, the same conceptual approach is applied to sequences. As we have one run, sequences were only conmbined accros V regions. **In case of more than one run, previous combining accros runs would be needed.**


In [ ]:
%%bash

qiime feature-table merge-seqs \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V2/dada2_pyro/dada2_pyro_V2_rep_seq.qza \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V3/dada2_pyro/dada2_pyro_V3_rep_seq.qza \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V4/dada2_pyro/dada2_pyro_V4_rep_seq.qza \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V67/dada2_pyro/dada2_pyro_V67_rep_seq.qza \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V8/dada2_pyro/dada2_pyro_V8_rep_seq.qza \
    --i-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/V9/dada2_pyro/dada2_pyro_V9_rep_seq.qza \
    --o-merged-data /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_rep_seqs.qza

In [ ]:
%%bash

qiime metadata tabulate \
    --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_rep_seqs.qza \
    --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_rep_seqs.qzv 

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_rep_seqs.qzv 

#### 9. Classify taxonomy.


Database OTU fasta and taxonomy files (SILVA 138) were directly downloaded from from Qiime2 as artifacts and saved in the foder used for classification files. In this case, the **full-lenght** sequences and taxonomy are used because we are going to classify sequences comming from all V regions.

There is an option to extract the V region of interest, but since the sequences primers are needed to do this and Ion Torrent has them as propietary, this step can't be developed.

In this case, the path to follow could be:

1. To use the full-length sequences and taxonomy for each V region (feature table and sequences) to obtain results for each V region.

2. To use the full-length sequences and taxonomy for each the consensus results (feature table and sequences) to obtain results for all regions together.

##### 9.1 Classify taxonomy for consensus results.

In [ ]:
%%bash

qiime feature-classifier classify-consensus-vsearch \
    --i-query /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/merged_rep_seqs.qza \
    --i-reference-reads /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/silva-138-99-seqs.qza \
    --i-reference-taxonomy /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/silva_classifier/silva-138-99-tax.qza \
    --p-strand 'plus' \
    --p-threads 2 \
    --o-classification /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/full_silva_taxonomy.qza \
    --verbose

In [ ]:
%%bash

qiime metadata tabulate \
  --m-input-file /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/full_silva_taxonomy.qza  \
  --o-visualization /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/full_silva_taxonomy.qzv

In [ ]:
! qiime tools view /home/andres/Escritorio/bioinfo_hematology_unit/myeloma_microbiome_project_2022/pipe_results/full_silva_taxonomy.qzv

Filter feature table (use tesis alba ipynb)

Barplot

##### 9.2 Classify taxonomy for each V region individually.